In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install opencv-contrib-python lap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:00


In [10]:
import cv2
import os
import glob
import numpy as np
import time
import pandas as pd
from collections import defaultdict
from lap import lapjv
from google.colab.patches import cv2_imshow

In [11]:
def iou(bb1, bb2):
    x1 = max(bb1[0], bb2[0])
    y1 = max(bb1[1], bb2[1])
    x2 = min(bb1[0] + bb1[2], bb2[0] + bb2[2])
    y2 = min(bb1[1] + bb1[3], bb2[1] + bb2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    bb1_area = bb1[2] * bb1[3]
    bb2_area = bb2[2] * bb2[3]
    union_area = bb1_area + bb2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0


In [16]:
def run_kcf_on_sequence(seq_path):
    img_dir = os.path.join(seq_path, "img1")
    gt_path = os.path.join(seq_path, "gt/gt.txt")

    img_files = sorted(glob.glob(os.path.join(img_dir, '*.jpg')))
    gt = pd.read_csv(gt_path, header=None).values

    trackers = {}
    results = []
    start_time = time.time()

    for frame_idx, img_file in enumerate(img_files, start=1):
        frame = cv2.imread(img_file)
        detections = gt[gt[:, 0] == frame_idx]

        # Initialize trackers on first frame
        if frame_idx == 1:
            for det in detections:
                tid = int(det[1])
                bbox = tuple(map(int, det[2:6]))
                tracker = cv2.TrackerKCF_create()
                tracker.init(frame, bbox)
                trackers[tid] = tracker

        # Update all trackers
        frame_results = []
        for tid, tracker in list(trackers.items()):
            ok, bbox = tracker.update(frame)
            if ok:
                x, y, w, h = map(int, bbox)
                frame_results.append((frame_idx, tid, x, y, w, h))

                if frame_idx % 200 == 0:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, f'ID {tid}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            else:
                del trackers[tid]

        results.extend(frame_results)

        if frame_idx % 200 == 0:
            cv2_imshow(frame)

    elapsed = time.time() - start_time
    fps = len(img_files) / elapsed
    return results, gt, fps


In [25]:
def evaluate_mot(gt, pred):
    gt_by_frame = defaultdict(list)
    pred_by_frame = defaultdict(list)

    for row in gt:
        gt_by_frame[int(row[0])].append((int(row[1]), list(map(float, row[2:6]))))

    for row in pred:
        pred_by_frame[int(row[0])].append((int(row[1]), list(map(float, row[2:6]))))

    mota_total, motp_total, total_matches = 0, 0, 0
    misses, false_positives, mismatches = 0, 0, 0
    total_gt = 0

    for f in sorted(gt_by_frame.keys()):
        gts = gt_by_frame[f]
        preds = pred_by_frame.get(f, [])

        if len(gts) == 0 and len(preds) == 0:
            continue

        if len(gts) == 0:
            false_positives += len(preds)
            continue

        if len(preds) == 0:
            misses += len(gts)
            total_gt += len(gts)
            continue

        cost_matrix = np.ones((len(gts), len(preds))) * 1e9
        for i, (_, g) in enumerate(gts):
            for j, (_, p) in enumerate(preds):
                cost_matrix[i, j] = 1 - iou(g, p)

        matched_pairs = np.argwhere(cost_matrix < 0.5)

        used_gt = set()
        used_pred = set()
        matched = 0

        for i, j in matched_pairs:
            if i in used_gt or j in used_pred:
                continue
            used_gt.add(i)
            used_pred.add(j)
            motp_total += 1 - cost_matrix[i, j]
            matched += 1

        total_matches += matched
        total_gt += len(gts)
        misses += len(gts) - matched
        false_positives += len(preds) - matched

    mota = 1 - (misses + false_positives + mismatches) / total_gt if total_gt > 0 else 0
    motp = motp_total / total_matches if total_matches > 0 else 0
    return mota, motp


In [21]:
sequence_path = '/content/drive/MyDrive/CMP719/MOT17/MOT17-02-SDP/'
sequence_path2 = '/content/drive/MyDrive/CMP719/MOT17/MOT17-04-FRCNN/'
sequence_path3 = '/content/drive/MyDrive/CMP719/MOT17/MOT17-09-DPM/'
sequence_path4 = '/content/drive/MyDrive/CMP719/MOT17/MOT17-10-SDP/'

In [28]:
results, gt, fps = run_kcf_on_sequence(sequence_path)
mota, motp = evaluate_mot(gt, results)

print(f"Results for: MOT17-02-SDP")
print(f"MOTA: {mota:.4f}")
print(f"MOTP: {motp:.4f}")
print(f"FPS: {fps:.2f}")

Output hidden; open in https://colab.research.google.com to view.

In [29]:
results, gt, fps = run_kcf_on_sequence(sequence_path2)
mota, motp = evaluate_mot(gt, results)

print(f"Results for: MOT17-04-FRCNN")
print(f"MOTA: {mota:.4f}")
print(f"MOTP: {motp:.4f}")
print(f"FPS: {fps:.2f}")

Output hidden; open in https://colab.research.google.com to view.

In [30]:
results, gt, fps = run_kcf_on_sequence(sequence_path3)
mota, motp = evaluate_mot(gt, results)

print(f"Results for: MOT17-09-DPM")
print(f"MOTA: {mota:.4f}")
print(f"MOTP: {motp:.4f}")
print(f"FPS: {fps:.2f}")

Output hidden; open in https://colab.research.google.com to view.

In [31]:
results, gt, fps = run_kcf_on_sequence(sequence_path4)
mota, motp = evaluate_mot(gt, results)

print(f"Results for: MOT17-10-SDP")
print(f"MOTA: {mota:.4f}")
print(f"MOTP: {motp:.4f}")
print(f"FPS: {fps:.2f}")

Output hidden; open in https://colab.research.google.com to view.